## 네이버 뉴스 수집기
---
### 사용한 버젼

In [178]:
# %load_ext watermark
%watermark -adatacampus-ku -d -v --iversions

Author: datacampus-ku

Python implementation: CPython
Python version       : 3.7.9
IPython version      : 7.20.0

json   : 2.0.9
urllib3: 1.26.3
parser : 0.5
re     : 2.2.1



In [1]:
import json
import re
import urllib.request
from urllib.parse import urlparse, parse_qs
from bs4 import BeautifulSoup
from requests import request, HTTPError
from requests.compat import urljoin, urlparse, quote, unquote
from requests.sessions import Session
from time import sleep

In [2]:
from selenium.webdriver import Chrome, ChromeOptions

In [ ]:
options = ChromeOptions()
options.add_argument("user-agent=Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.87 Safari/537.36")
driver = Chrome(options=options)

headers = {
    'user-agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.87 Safari/537.36'
}

In [16]:
def download(url, params={}, headers=headers, method='GET', maximum=2):
    resp = request(url=url, params=params if method=='GET' else {},
            data=params if method=='POST' else {},
            headers=headers, method=method)
    
    
    try:
        resp.raise_for_status()
    except HTTPError as e:
        if 500 <= e.response.status_code < 600 and maximum > 0:
            print('retry')
            sleep(1) # 1초?
            resp = download(url, params, headers, method, maximum-1) # inf loop
        else:
            print(e)
        
    return resp

In [9]:
def can_fetch(url, path):
    resp = download(urljoin(url, '/robots.txt'))
    allow = list()
    disallow = list()
    
    
    if resp.status_code == 200:
        useragent = '*'
        for _ in resp.text.splitlines():
            pair = _.split(':')
            if len(pair) == 2:
                if pair[0].strip() == 'User-agent':
                    useragent = pair[1]
                else:
                    if pair[1].strip() == 'Allow':
                        allow.append(pair[1].strip())
                    else:
                        disallow.append(pair[1].strip())
      
    return True if path in allow else False, allow, disallow

In [13]:
def download_video(url):
    save_file = './MOV/temp.mp4'
    urllib.request.urlretrieve(url, save_file)
    
    return True

In [27]:
def cleaning(data):
    data = re.sub(r'오류를 우회하기 위한 함수 추가', '', data)
    # 안지워질 때를 대비
    data = re.sub(r'\b[a-z_][a-zA-Z_()]{4,}\b', '', data)
    # 1. 사이드 스페이스가 있을 때 반드시 소문자+_  1글자로 시작, 영문(대소문자 포함) 4글자 이상 삭제 => 제일 뒷줄의 javascript 제거
    data = re.sub(r'\b[(){}/]{2,}', '', data)
    # 2. 함수 기호 삭제 => () {} 제거
    data = re.sub(r'([.,?!])(\w)', '\g<1> \g<2>', data)
    # 3. 구두점 다음에 글자가 있는 경우 공백 추가 => .단어 => . 단어로 변경
    data = re.sub(r'\s{2,}', '\n', data).strip()
    # 4. 공백 2개 이상 반복되는 경우 한 개만 => \n\n\n\n\n 또는      =>\n 또는 
    data = re.sub(r'[^\s\w\d.,?!:\'\"]', '', data)
    # 5. 공백, 문자, 숫자, 구두점을 제외한 나머지 글자 삭제 => 모든 특수문자 제거
    data ='\n'.join(data.splitlines()[1:])
    # * 네이버에서 수집한 뉴스는 첫 줄에 주석이 달려있음 => 첫 줄 주석 삭제
    return data

In [6]:
# 찾을 뉴스 URL
url = "https://news.naver.com/main/tvgallery/replay.naver?sid=751&date=20200101#052_0001383422"

# 동영상을 다운받을 서버
base_url = "https://apis.naver.com/rmcnmv/rmcnmv/vod/play/v2.0"

In [57]:
down = download(url=url, method='POST', headers=headers)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [39]:
# Network에서 XHR로 통신하는 링크 : 
request_url = "https://apis.naver.com/rmcnmv/rmcnmv/vod/play/v2.0/BAA1BD447C6AEAAAFC86174FC5C8BA8DCF46?key=V125b02e82c56f176158790ee53ef26e33e09d79e50772ed27e1b5d2aa1623c99302a90ee53ef26e33e09&pid=rmcPlayer_16269668385546947&sid=2006&ver=2.0&devt=html5_mo&doct=json&ptc=https&sptc=https&cpt=vtt&ctls=%7B%22visible%22%3A%7B%22fullscreen%22%3Atrue%2C%22logo%22%3Afalse%2C%22playbackRate%22%3Afalse%2C%22scrap%22%3Afalse%2C%22playCount%22%3Atrue%2C%22commentCount%22%3Atrue%2C%22title%22%3Atrue%2C%22writer%22%3Atrue%2C%22expand%22%3Afalse%2C%22subtitles%22%3Atrue%2C%22thumbnails%22%3Atrue%2C%22quality%22%3Atrue%2C%22setting%22%3Atrue%2C%22script%22%3Afalse%2C%22logoDimmed%22%3Atrue%2C%22badge%22%3Atrue%2C%22seekingTime%22%3Atrue%2C%22muted%22%3Atrue%2C%22muteButton%22%3Afalse%2C%22viewerNotice%22%3Afalse%2C%22linkCount%22%3Afalse%2C%22createTime%22%3Afalse%2C%22thumbnail%22%3Atrue%7D%2C%22clicked%22%3A%7B%22expand%22%3Afalse%2C%22subtitles%22%3Afalse%7D%7D&pv=4.18.40&dr=2560x1440&lc=ko_KR&videoId=BAA1BD447C6AEAAAFC86174FC5C8BA8DCF46"

In [58]:
# data에 parameter들을 나눠서 저장
keys = re.findall(r'\?(key=.+)', request_url)[0]
data = parse_qs(urljoin(base=base_url, url=''.join(['&', keys])))

In [59]:
keys

'key=V125b02e82c56f176158790ee53ef26e33e09d79e50772ed27e1b5d2aa1623c99302a90ee53ef26e33e09&pid=rmcPlayer_16269668385546947&sid=2006&ver=2.0&devt=html5_mo&doct=json&ptc=https&sptc=https&cpt=vtt&ctls=%7B%22visible%22%3A%7B%22fullscreen%22%3Atrue%2C%22logo%22%3Afalse%2C%22playbackRate%22%3Afalse%2C%22scrap%22%3Afalse%2C%22playCount%22%3Atrue%2C%22commentCount%22%3Atrue%2C%22title%22%3Atrue%2C%22writer%22%3Atrue%2C%22expand%22%3Afalse%2C%22subtitles%22%3Atrue%2C%22thumbnails%22%3Atrue%2C%22quality%22%3Atrue%2C%22setting%22%3Atrue%2C%22script%22%3Afalse%2C%22logoDimmed%22%3Atrue%2C%22badge%22%3Atrue%2C%22seekingTime%22%3Atrue%2C%22muted%22%3Atrue%2C%22muteButton%22%3Afalse%2C%22viewerNotice%22%3Afalse%2C%22linkCount%22%3Afalse%2C%22createTime%22%3Afalse%2C%22thumbnail%22%3Atrue%7D%2C%22clicked%22%3A%7B%22expand%22%3Afalse%2C%22subtitles%22%3Afalse%7D%7D&pv=4.18.40&dr=2560x1440&lc=ko_KR&videoId=BAA1BD447C6AEAAAFC86174FC5C8BA8DCF46'

In [60]:
data

{'key': ['V125b02e82c56f176158790ee53ef26e33e09d79e50772ed27e1b5d2aa1623c99302a90ee53ef26e33e09'],
 'pid': ['rmcPlayer_16269668385546947'],
 'sid': ['2006'],
 'ver': ['2.0'],
 'devt': ['html5_mo'],
 'doct': ['json'],
 'ptc': ['https'],
 'sptc': ['https'],
 'cpt': ['vtt'],
 'ctls': ['{"visible":{"fullscreen":true,"logo":false,"playbackRate":false,"scrap":false,"playCount":true,"commentCount":true,"title":true,"writer":true,"expand":false,"subtitles":true,"thumbnails":true,"quality":true,"setting":true,"script":false,"logoDimmed":true,"badge":true,"seekingTime":true,"muted":true,"muteButton":false,"viewerNotice":false,"linkCount":false,"createTime":false,"thumbnail":true},"clicked":{"expand":false,"subtitles":false}}'],
 'pv': ['4.18.40'],
 'dr': ['2560x1440'],
 'lc': ['ko_KR'],
 'videoId': ['BAA1BD447C6AEAAAFC86174FC5C8BA8DCF46']}

In [44]:
# 동영상 링크 주소를 얻기 위해 GET
respond = download(''.join([base_url, '/', data['videoId'][0], '?', keys]))

In [50]:
respond.json()

{'errorCode': 'EXPIRED_KEY',
 'errorMessage': 'The server connection is unstable.<br>Press the F5 key or click the Refresh button<br>and try again',
 'countryCode': 'KR'}

In [49]:
rmcPlayer = json.dumps(respond.json()['videos'])

KeyError: 'videos'

In [163]:
# 최종 뉴스 720P 영상 링크
video_link = re.findall(r'"720P".+?"source": "(.+)",', rmcPlayer)[0]

In [4]:
# 다운로드 속도가 생각보다 많이 느리다.. 31MB를 다운받는데 20초 걸린듯?
download_video(video_link)

NameError: name 'download_video' is not defined

In [28]:
news_url = 'https://news.naver.com/main/read.naver?mode=LPOSD&mid=tvh&sid1=121&sid2=355&oid=056&aid=0011089474'

resp = download(news_url)

# Scraping 영역 --> 
dom = BeautifulSoup(resp.text, 'lxml')

# 헤드라인만 가져오게 됨.
main_area = dom.select_one('#main_content') # 찾지 못할 시 None이 들어가게 됨.

# li의 자식 중 a[href], li의 자식 중 div의 자식 중 a[href] : 이 2형태로 처리
news_list = main_area.select('li > a[href], li > div > a[href]')

article_area = dom.select_one('#articleBodyContents') # 이게 있으면 뉴스, 아니라면 건너뜀

if article_area:
    file_name = re.sub(r'\W', '', news_url.split('/')[-1]) # \W ASCII가 아닌 비표준 문자.

with open('TXT/'+file_name+'.txt', 'w', encoding='utf-8') as f: # TXT 폴더에 저장
    f.write(cleaning(article_area.text))